# जनरेटिव नेटवर्क्स

रिकरेंट न्यूरल नेटवर्क्स (RNNs) और उनके गेटेड सेल वेरिएंट्स जैसे लॉन्ग शॉर्ट टर्म मेमोरी सेल्स (LSTMs) और गेटेड रिकारेंट यूनिट्स (GRUs) ने भाषा मॉडलिंग के लिए एक तंत्र प्रदान किया, यानी वे शब्दों के क्रम को सीख सकते हैं और अनुक्रम में अगले शब्द की भविष्यवाणी कर सकते हैं। यह हमें RNNs का उपयोग **जनरेटिव कार्यों** के लिए करने की अनुमति देता है, जैसे साधारण टेक्स्ट जनरेशन, मशीन ट्रांसलेशन, और यहां तक कि इमेज कैप्शनिंग।

पिछली यूनिट में हमने जिस RNN आर्किटेक्चर पर चर्चा की थी, उसमें प्रत्येक RNN यूनिट ने अगले हिडन स्टेट को आउटपुट के रूप में उत्पन्न किया। हालांकि, हम प्रत्येक रिकारेंट यूनिट में एक और आउटपुट जोड़ सकते हैं, जो हमें एक **अनुक्रम** आउटपुट करने की अनुमति देगा (जो मूल अनुक्रम की लंबाई के बराबर होगा)। इसके अलावा, हम ऐसे RNN यूनिट्स का उपयोग कर सकते हैं जो प्रत्येक चरण में इनपुट स्वीकार नहीं करते, बल्कि केवल एक प्रारंभिक स्टेट वेक्टर लेते हैं और फिर आउटपुट का एक अनुक्रम उत्पन्न करते हैं।

इस नोटबुक में, हम सरल जनरेटिव मॉडलों पर ध्यान केंद्रित करेंगे जो हमें टेक्स्ट जनरेट करने में मदद करते हैं। सरलता के लिए, चलिए **कैरेक्टर-लेवल नेटवर्क** बनाते हैं, जो अक्षर दर अक्षर टेक्स्ट जनरेट करता है। प्रशिक्षण के दौरान, हमें कुछ टेक्स्ट कॉर्पस लेना होगा और इसे अक्षर अनुक्रमों में विभाजित करना होगा।


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## वर्णमाला शब्दावली बनाना

वर्ण-स्तरीय जनरेटिव नेटवर्क बनाने के लिए, हमें टेक्स्ट को शब्दों के बजाय व्यक्तिगत अक्षरों में विभाजित करना होगा। यह एक अलग टोकनाइज़र को परिभाषित करके किया जा सकता है:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


आइए देखें कि हम अपने डेटासेट से टेक्स्ट को कैसे एन्कोड कर सकते हैं:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## जनरेटिव RNN को प्रशिक्षित करना

हम RNN को टेक्स्ट जनरेट करने के लिए इस प्रकार प्रशिक्षित करेंगे। हर चरण में, हम `nchars` लंबाई के अक्षरों की एक श्रृंखला लेंगे और नेटवर्क से प्रत्येक इनपुट अक्षर के लिए अगला आउटपुट अक्षर जनरेट करने के लिए कहेंगे:

![चित्र 'HELLO' शब्द के RNN जनरेशन का उदाहरण दिखा रहा है।](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

वास्तविक परिदृश्य के आधार पर, हम कुछ विशेष अक्षरों को भी शामिल करना चाह सकते हैं, जैसे *end-of-sequence* `<eos>`। हमारे मामले में, हम नेटवर्क को अंतहीन टेक्स्ट जनरेशन के लिए प्रशिक्षित करना चाहते हैं, इसलिए हम प्रत्येक श्रृंखला का आकार `nchars` टोकन के बराबर तय करेंगे। परिणामस्वरूप, प्रत्येक प्रशिक्षण उदाहरण में `nchars` इनपुट और `nchars` आउटपुट होंगे (जो इनपुट श्रृंखला को एक प्रतीक बाईं ओर शिफ्ट करके प्राप्त किए जाते हैं)। मिनीबैच में ऐसी कई श्रृंखलाएं शामिल होंगी।

हम मिनीबैच को इस प्रकार जनरेट करेंगे कि प्रत्येक समाचार टेक्स्ट जिसकी लंबाई `l` है, से सभी संभावित इनपुट-आउटपुट संयोजन बनाएंगे (ऐसे `l-nchars` संयोजन होंगे)। ये एक मिनीबैच बनाएंगे, और प्रत्येक प्रशिक्षण चरण में मिनीबैच का आकार अलग-अलग होगा।


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

अब हम जनरेटर नेटवर्क को परिभाषित करते हैं। यह किसी भी पुनरावर्ती सेल पर आधारित हो सकता है जिसे हमने पिछले यूनिट में चर्चा की थी (सिंपल, LSTM या GRU)। हमारे उदाहरण में, हम LSTM का उपयोग करेंगे।

चूंकि नेटवर्क अक्षरों को इनपुट के रूप में लेता है और शब्दावली का आकार काफी छोटा है, हमें एम्बेडिंग लेयर की आवश्यकता नहीं है। वन-हॉट-एनकोडेड इनपुट सीधे LSTM सेल में जा सकता है। हालांकि, क्योंकि हम अक्षरों के नंबर को इनपुट के रूप में पास करते हैं, हमें उन्हें LSTM में पास करने से पहले वन-हॉट-एनकोड करना होगा। यह `forward` पास के दौरान `one_hot` फ़ंक्शन को कॉल करके किया जाता है। आउटपुट एनकोडर एक लीनियर लेयर होगा जो हिडन स्टेट को वन-हॉट-एनकोडेड आउटपुट में बदल देगा।


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

प्रशिक्षण के दौरान, हम उत्पन्न किए गए टेक्स्ट का नमूना लेना चाहते हैं। इसे करने के लिए, हम `generate` फ़ंक्शन को परिभाषित करेंगे, जो प्रारंभिक स्ट्रिंग `start` से शुरू करते हुए, लंबाई `size` का आउटपुट स्ट्रिंग उत्पन्न करेगा।

इसका काम करने का तरीका निम्नलिखित है। सबसे पहले, हम पूरी प्रारंभिक स्ट्रिंग को नेटवर्क के माध्यम से पास करेंगे, और आउटपुट स्थिति `s` और अगला अनुमानित अक्षर `out` प्राप्त करेंगे। चूंकि `out` वन-हॉट एन्कोडेड होता है, हम `argmax` का उपयोग करके शब्दावली में अक्षर `nc` का इंडेक्स प्राप्त करेंगे, और `itos` का उपयोग करके वास्तविक अक्षर का पता लगाएंगे और इसे अक्षरों की परिणामी सूची `chars` में जोड़ देंगे। इस प्रक्रिया को एक अक्षर उत्पन्न करने के लिए `size` बार दोहराया जाता है, ताकि आवश्यक संख्या में अक्षर उत्पन्न किए जा सकें।


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

अब चलिए प्रशिक्षण शुरू करते हैं! प्रशिक्षण लूप लगभग हमारे सभी पिछले उदाहरणों जैसा ही है, लेकिन सटीकता (accuracy) के बजाय हम हर 1000 epochs पर उत्पन्न किया गया नमूना टेक्स्ट प्रिंट करते हैं।

विशेष ध्यान उस तरीके पर देना होगा जिससे हम हानि (loss) की गणना करते हैं। हमें हानि की गणना एक-हॉट-एन्कोडेड आउटपुट `out` और अपेक्षित टेक्स्ट `text_out` (जो कि कैरेक्टर इंडेक्स की सूची है) के आधार पर करनी होगी। सौभाग्य से, `cross_entropy` फ़ंक्शन अननॉर्मलाइज़्ड नेटवर्क आउटपुट को पहले तर्क के रूप में और क्लास नंबर को दूसरे तर्क के रूप में अपेक्षित करता है, जो कि हमारे पास पहले से ही है। यह मिनीबैच साइज पर स्वचालित औसत भी करता है।

हम प्रशिक्षण को `samples_to_train` सैंपल्स तक सीमित करते हैं, ताकि बहुत अधिक समय न लगे। हम आपको प्रोत्साहित करते हैं कि आप प्रयोग करें और लंबे समय तक प्रशिक्षण आज़माएं, संभवतः कई epochs के लिए (ऐसे मामले में आपको इस कोड के चारों ओर एक और लूप बनाना होगा)।


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

यह उदाहरण पहले से ही काफी अच्छा टेक्स्ट उत्पन्न करता है, लेकिन इसे कई तरीकों से और बेहतर बनाया जा सकता है:

* **बेहतर मिनीबैच जनरेशन**। जिस तरीके से हमने ट्रेनिंग के लिए डेटा तैयार किया, वह था एक सैंपल से एक मिनीबैच बनाना। यह आदर्श नहीं है, क्योंकि मिनीबैच अलग-अलग आकार के होते हैं, और कुछ तो बनाए भी नहीं जा सकते, क्योंकि टेक्स्ट `nchars` से छोटा होता है। इसके अलावा, छोटे मिनीबैच GPU को पर्याप्त रूप से लोड नहीं करते। यह अधिक समझदारी होगी कि सभी सैंपल से एक बड़ा टेक्स्ट का हिस्सा लिया जाए, फिर सभी इनपुट-आउटपुट जोड़े बनाए जाएं, उन्हें शफल किया जाए, और समान आकार के मिनीबैच बनाए जाएं।

* **मल्टीलायर LSTM**। 2 या 3 लेयर के LSTM सेल्स को आज़माना समझदारी होगी। जैसा कि हमने पिछले यूनिट में बताया था, LSTM की प्रत्येक लेयर टेक्स्ट से कुछ विशेष पैटर्न निकालती है, और कैरेक्टर-लेवल जनरेटर के मामले में हम उम्मीद कर सकते हैं कि निचली LSTM लेयर अक्षरों के समूह (syllables) को निकालने के लिए जिम्मेदार होगी, और ऊपरी लेयर शब्दों और शब्द संयोजनों के लिए। इसे आसानी से LSTM कंस्ट्रक्टर में लेयर की संख्या का पैरामीटर पास करके लागू किया जा सकता है।

* आप **GRU यूनिट्स** के साथ भी प्रयोग कर सकते हैं और देख सकते हैं कि कौन से बेहतर प्रदर्शन करते हैं, और **अलग-अलग हिडन लेयर साइज** के साथ भी। बहुत बड़ी हिडन लेयर ओवरफिटिंग का कारण बन सकती है (जैसे कि नेटवर्क सटीक टेक्स्ट सीख लेगा), और छोटी साइज अच्छे परिणाम नहीं दे सकती।


## सॉफ्ट टेक्स्ट जनरेशन और टेम्परेचर

`generate` की पिछली परिभाषा में, हम हमेशा उस कैरेक्टर को अगला कैरेक्टर चुनते थे जिसकी संभावना सबसे अधिक होती थी। इसका परिणाम यह होता था कि टेक्स्ट अक्सर बार-बार एक ही कैरेक्टर सीक्वेंस में "चक्रित" हो जाता था, जैसे इस उदाहरण में:
```
today of the second the company and a second the company ...
```

हालांकि, अगर हम अगले कैरेक्टर के लिए संभावना वितरण को देखें, तो यह हो सकता है कि कुछ उच्चतम संभावनाओं के बीच का अंतर बहुत बड़ा न हो, जैसे कि एक कैरेक्टर की संभावना 0.2 हो, और दूसरे की 0.19। उदाहरण के लिए, जब हम '*play*' सीक्वेंस में अगले कैरेक्टर की तलाश कर रहे हों, तो अगला कैरेक्टर स्पेस या **e** (जैसे शब्द *player* में) दोनों ही हो सकते हैं।

इससे यह निष्कर्ष निकलता है कि हमेशा उच्चतम संभावना वाले कैरेक्टर को चुनना "न्यायसंगत" नहीं है, क्योंकि दूसरे उच्चतम को चुनने से भी अर्थपूर्ण टेक्स्ट बन सकता है। यह अधिक समझदारी होगी कि नेटवर्क आउटपुट द्वारा दी गई संभावना वितरण से **सैंपल** करके कैरेक्टर चुने जाएं।

यह सैंपलिंग `multinomial` फंक्शन का उपयोग करके की जा सकती है, जो तथाकथित **मल्टिनोमियल वितरण** को लागू करता है। एक फंक्शन जो इस **सॉफ्ट** टेक्स्ट जनरेशन को लागू करता है, नीचे परिभाषित है:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

हमने एक और पैरामीटर **तापमान** पेश किया है, जिसका उपयोग यह संकेत देने के लिए किया जाता है कि हमें उच्चतम संभावना से कितनी दृढ़ता से चिपकना चाहिए। यदि तापमान 1.0 है, तो हम निष्पक्ष बहुपद नमूना लेते हैं, और जब तापमान अनंत तक जाता है - सभी संभावनाएँ समान हो जाती हैं, और हम अगला वर्ण यादृच्छिक रूप से चुनते हैं। नीचे दिए गए उदाहरण में हम देख सकते हैं कि जब हम तापमान को बहुत अधिक बढ़ाते हैं तो पाठ अर्थहीन हो जाता है, और जब यह 0 के करीब होता है तो यह "चक्रित" कठोर-जनित पाठ जैसा दिखता है।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को आधिकारिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
